In [2]:
import pandas as pd
df = pd.read_csv('batch_results.csv')

In [4]:
grouped = df.groupby(['RunId', 'iteration'])

In [5]:
def find_steady_state(subdf):
    # Find the first step where majority_action and stick_in_the_muds stop changing
    maj = subdf['majority_action'].values
    sm = subdf['stick_in_the_muds'].values
    for i in range(1, len(subdf)):
        if all(maj[i:] == maj[i]) and all(sm[i:] == sm[i]):
            return i  # NI: step when steady state starts
    return len(subdf) - 1  # If never steady, return last step

ni_results = grouped.apply(find_steady_state)
ni_results

/var/folders/_y/p8r1pq5s491gjhlr2x0pt9mc0000gn/T/ipykernel_24440/2807331689.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ni_results = grouped.apply(find_steady_state)


RunId  iteration
0      0            20
1      0            22
2      0            22
3      0            16
4      0            12
                    ..
1795   99            5
1796   99            5
1797   99            4
1798   99            3
1799   99            4
Length: 1800, dtype: int64

In [6]:
def get_sm(subdf):
    ni = find_steady_state(subdf)
    return subdf.iloc[ni]['stick_in_the_muds']

sm_results = grouped.apply(get_sm)
sm_results

/var/folders/_y/p8r1pq5s491gjhlr2x0pt9mc0000gn/T/ipykernel_24440/1764621027.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sm_results = grouped.apply(get_sm)


RunId  iteration
0      0            0.0
1      0            0.0
2      0            0.0
3      0            0.0
4      0            0.0
                   ... 
1795   99           0.0
1796   99           0.0
1797   99           0.0
1798   99           0.0
1799   99           0.0
Length: 1800, dtype: float64

In [7]:
def get_ie(subdf):
    ni = find_steady_state(subdf)
    initial_majority = subdf.iloc[0]['majority_action']
    steady_majority = subdf.iloc[ni:]['majority_action']
    matches = (steady_majority == initial_majority).sum()
    return matches / len(steady_majority) if len(steady_majority) > 0 else 0

ie_results = grouped.apply(get_ie)
ie_results

/var/folders/_y/p8r1pq5s491gjhlr2x0pt9mc0000gn/T/ipykernel_24440/3198603246.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ie_results = grouped.apply(get_ie)


RunId  iteration
0      0            1.0
1      0            1.0
2      0            1.0
3      0            1.0
4      0            1.0
                   ... 
1795   99           1.0
1796   99           1.0
1797   99           1.0
1798   99           1.0
1799   99           1.0
Length: 1800, dtype: float64

In [10]:
results = pd.DataFrame({
    'NI': ni_results,
    'SM': sm_results,
    'IE': ie_results
}).reset_index()
results

,RunId,iteration,NI,SM,IE
0,0,0,20,0.0,1.0
1,1,0,22,0.0,1.0
2,2,0,22,0.0,1.0
3,3,0,16,0.0,1.0
4,4,0,12,0.0,1.0
...,...,...,...,...,...
1795,1795,99,5,0.0,1.0
1796,1796,99,5,0.0,1.0
1797,1797,99,4,0.0,1.0
1798,1798,99,3,0.0,1.0
